In [ ]:
# 1. No, they would return the same gradients, makes learning impossible

In [ ]:
# 2. Yes

In [ ]:
# 3. No 0/dead neurons, average output at 0 negates the vanishing gradients problem, self normalizing effect for networks with all dense layers

In [ ]:
# 4.
# SELU: networks with all dense layers
# leaky ReLU(s): everything else
# ReLU: simplicity
# tanh: for output layers with result -1 to 1
# logistic: for output layers with probability-like output
# softmax: for output layers with mutually exclusive results

In [ ]:
# 5. High momentum can cause it to roll pass optimum

In [ ]:
# 6. l1 regularization, manual forcing weights to 0, and tensorflow optimization all can produce sparse model

In [ ]:
# 7. Yes, it equals to training many networks. No
# Yes, it equals to predicting with many networks

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
# 8. Deep neural net with CIFAR 10

In [3]:
# Import dataset
from tensorflow.keras.datasets import cifar10

(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()
X_train_full.shape

(50000, 32, 32, 3)

In [4]:
X_valid, X_train = X_train_full[:10000], X_train_full[10000:]
y_valid, y_train = y_train_full[:10000], y_train_full[10000:]

In [5]:
# Scaling for non SELU activations
X_valid = X_valid.astype('float32') / 255
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [26]:
# Model architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization

model = Sequential()
model.add(Flatten(input_shape=[32, 32, 3]))
for i in range(0, 20):
    if i > 15 and i <19:
        model.add(Dropout(rate=0.25))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='elu', kernel_initializer='lecun_normal'))
model.add(Dense(10, activation='softmax'))

In [27]:
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

In [28]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 3072)              0         
_________________________________________________________________
batch_normalization_40 (Batc (None, 3072)              12288     
_________________________________________________________________
dense_42 (Dense)             (None, 100)               307300    
_________________________________________________________________
batch_normalization_41 (Batc (None, 100)               400       
_________________________________________________________________
dense_43 (Dense)             (None, 100)               10100     
_________________________________________________________________
batch_normalization_42 (Batc (None, 100)               400       
_________________________________________________________________
dense_44 (Dense)             (None, 100)              

In [10]:
# define callback
from tensorflow.keras.callbacks import EarlyStopping

early_cb = EarlyStopping(patience=10, restore_best_weights=True)

In [30]:
# 1 cycle scheduling learning rate
K = keras.backend

class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                last_iter=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate/10
        self.last_iterations = last_iter or iterations//10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate/1000
        self.iteration = 0
    
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return((rate2-rate1) * (self.iteration-iter1) / (iter2-iter1) + rate1)
    
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, 
                                     self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2*self.half_iteration,
                                    self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2*self.half_iteration, self.iterations,
                                    self.start_rate, self.last_rate)
            rate = max(rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.lr, rate)   

In [31]:
n_epochs = 100
batch_size = 32

In [32]:
onecycle = OneCycleScheduler(len(X_train) // batch_size*n_epochs, max_rate=0.01)

In [33]:
history = model.fit(X_train, y_train,
                   epochs=n_epochs,
                   batch_size=batch_size,
                   validation_data=(X_valid, y_valid),
                   callbacks=[onecycle])

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 27s 666us/sample - loss: 2.0014 - accuracy: 0.2748 - val_loss: 1.8791 - val_accuracy: 0.3408
Epoch 2/100
40000/40000 [==============================] - 21s 516us/sample - loss: 1.8077 - accuracy: 0.3490 - val_loss: 1.8257 - val_accuracy: 0.3530
Epoch 3/100
40000/40000 [==============================] - 21s 519us/sample - loss: 1.7307 - accuracy: 0.3856 - val_loss: 1.6864 - val_accuracy: 0.3981
Epoch 4/100
40000/40000 [==============================] - 20s 511us/sample - loss: 1.6839 - accuracy: 0.4022 - val_loss: 1.6293 - val_accuracy: 0.4125acy: 0.40
Epoch 5/100
40000/40000 [==============================] - 21s 516us/sample - loss: 1.6542 - accuracy: 0.4166 - val_loss: 1.6400 - val_accuracy: 0.4269- ETA: 
Epoch 6/100
40000/40000 [==============================] - 21s 518us/sample - loss: 1.6227 - accuracy: 0.4292 - val_loss: 1.5791 - val_accuracy: 0.4492
Epoch 7/100
40000/4000

40000/40000 [==============================] - 23s 569us/sample - loss: 1.1646 - accuracy: 0.6036 - val_loss: 11234351153292.0898 - val_accuracy: 0.4909
Epoch 53/100
40000/40000 [==============================] - 25s 622us/sample - loss: 1.1487 - accuracy: 0.6098 - val_loss: 20103315268574.4883 - val_accuracy: 0.5110
Epoch 54/100
40000/40000 [==============================] - 28s 708us/sample - loss: 1.1456 - accuracy: 0.6099 - val_loss: 59178778641930.0078 - val_accuracy: 0.5035
Epoch 55/100
40000/40000 [==============================] - 28s 712us/sample - loss: 1.1190 - accuracy: 0.6188 - val_loss: 2603672023236.1514 - val_accuracy: 0.5100
Epoch 56/100
40000/40000 [==============================] - 28s 709us/sample - loss: 1.1114 - accuracy: 0.6231 - val_loss: 230334857646.1261 - val_accuracy: 0.5054
Epoch 57/100
40000/40000 [==============================] - 28s 690us/sample - loss: 1.0988 - accuracy: 0.6251 - val_loss: 716883726415.4139 - val_accuracy: 0.5030
Epoch 58/100
40000/400

40000/40000 [==============================] - 29s 714us/sample - loss: 0.7772 - accuracy: 0.7347 - val_loss: 91168270785.2187 - val_accuracy: 0.5259


In [34]:
# MC Dropout prediction
y_probas = np.stack([model(X_test, training=True) for sample in range(100)])
y_proba = y_probas.mean(axis=0)
y_pred = np.argmax(y_proba, axis=1)
y_pred

array([3, 8, 0, ..., 3, 4, 7], dtype=int64)

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.5395